In [309]:
import os
from pathlib import Path
import sqlite3
import pandas as pd

pd.set_option('display.max_columns', 100)

In [310]:
# data directory and files

data_dir = Path.cwd().parents[1] / "data"
wzdx_data_file = data_dir / "wzdx.db"
inrix_data_dir = data_dir/ "INRIX data"
inrix_data_1 = inrix_data_dir / "I10-and-I17-1year" / "I10-and-I17-1year.csv"
inrix_data_2 = inrix_data_dir / "Loop101-1year" / "Loop101-1year.csv"
inrix_data_3 = inrix_data_dir / "SR60-1year" / "SR60-1year.csv"


In [311]:
# Defining methods to get data
# The name of the table in wzdx is events. Since this table is small, it can be loaded into the pandas dataframe directly w/o using too much memory.

def data_wzdx(data_file):
    con = sqlite3.connect(data_file)
    cur = con.cursor()

    cur.execute("SELECT * FROM events")
    columns = [desc[0] for desc in cur.description]

    data_df = pd.DataFrame(cur.fetchall(), columns=columns)

    con.close()

    return data_df

def data_inrix(data_file, n=1000):
    data = pd.read_csv(data_file, nrows=n)
    return data

In [312]:
import geopandas as gpd
from shapely.geometry import Point, LineString
import numpy as np
import polyline as ppl
from pyproj import Geod
import json
import folium

In [313]:
df = data_wzdx(wzdx_data_file)

def line_from_encoded(enc):
    if pd.isna(enc) or enc == '':
        return None
    coords_latlon = ppl.decode(enc)
    coords_lonlat = [(lon, lat) for (lat, lon) in coords_latlon]
    return LineString(coords_lonlat)

In [314]:
df.loc[df['Longitude'].isnull()]
df.loc[df['Latitude'].isnull()]

,ID,Organization,RoadwayName,DirectionOfTravel,Description,Reported,LastUpdated,StartDate,PlannedEndDate,LanesAffected,Latitude,Longitude,LatitudeSecondary,LongitudeSecondary,EventType,EventSubType,IsFullClosure,Severity,EncodedPolyline,Width,Height,Length,Weight,Speed,DetourPolyline,DetourInstructions,Recurrence,RecurrenceSchedules,Details,LaneCount
14649,458580,Tucson,USE CAUTION: FLOODING MANVILL ALL DIRECTIONS,None,USE CAUTION: FLOODING MANVILL ALL DIRECTIONS...,1752669174,1753199920,1752669174,NaN,No Data,NaN,NaN,NaN,NaN,accidentsAndIncidents,None,0,None,None,None,None,None,None,None,None,None,None,None,None,NaN
25593,482178,Tucson,ROAD CLOSED: MANN AVE & DREXEL RD WESTBOUND,None,ROAD CLOSED: MANN AVE & DREXEL RD WESTBOUND...,1755648142,1755716292,1755648142,NaN,No Data,NaN,NaN,NaN,NaN,accidentsAndIncidents,None,0,None,None,None,None,None,None,None,None,None,None,None,None,NaN
25594,482181,Tucson,ROAD CLOSED: DREXEL RD & WILMOT RD EASTBOUND,None,ROAD CLOSED: DREXEL RD & WILMOT RD EASTBOUND...,1755648437,1755716292,1755648437,NaN,No Data,NaN,NaN,NaN,NaN,accidentsAndIncidents,None,0,None,None,None,None,None,None,None,None,None,None,None,None,NaN


There are only 3 rows in WZDx data with no Latitude or Longitude. It is better to impute them.

In [315]:
df = df.loc[~df['Longitude'].isnull()]

Check for empty rows after imputing

In [316]:
df.loc[df['Longitude'].isnull()]
df.loc[df['Latitude'].isnull()]

,ID,Organization,RoadwayName,DirectionOfTravel,Description,Reported,LastUpdated,StartDate,PlannedEndDate,LanesAffected,Latitude,Longitude,LatitudeSecondary,LongitudeSecondary,EventType,EventSubType,IsFullClosure,Severity,EncodedPolyline,Width,Height,Length,Weight,Speed,DetourPolyline,DetourInstructions,Recurrence,RecurrenceSchedules,Details,LaneCount


In [317]:
time_columns = ['Reported', 'LastUpdated', 'StartDate', 'PlannedEndDate']

for col in time_columns:
    df['Local' + col] = df[col]%10000000000
    df['Local' + col] = pd.to_datetime(df['Local' + col], unit='s', utc=True).dt.tz_convert("America/Phoenix").dt.strftime("%Y-%m-%dT%H:%M:%S%z")


In [318]:
geom = []
# print(df.loc[df['EncodedPolyline'].isnull(),['EncodedPolyline', 'Longitude', 'Latitude']].sample(5)) , ppl.decode(df['EncodedPolyline'].iloc[14410])
df.loc[14410:14414]

,ID,Organization,RoadwayName,DirectionOfTravel,Description,Reported,LastUpdated,StartDate,PlannedEndDate,LanesAffected,Latitude,Longitude,LatitudeSecondary,LongitudeSecondary,EventType,EventSubType,IsFullClosure,Severity,EncodedPolyline,Width,Height,Length,Weight,Speed,DetourPolyline,DetourInstructions,Recurrence,RecurrenceSchedules,Details,LaneCount,LocalReported,LocalLastUpdated,LocalStartDate,LocalPlannedEndDate
14410,433569,ERS,N 51st Ave,South,Roadway closed on N 51st Ave Southbound from W...,1752901200,1751559673,1752901200,1.753099e+09,No Data,33.668974,-112.169390,33.667815,-112.169399,roadwork,Roadclosed,0,Major,a~nlEtaskTbC?|A@B?,None,None,None,None,None,None,None,"""<b>Mon:</b><br/>10:00 PM - 12:00 AM<br/><br/>...","[{""StartDate"": ""7/18/2025 10:00:00 PM-07:00:00...",Southbound 51st Ave at SR101L will be closed ...,2.0,2025-07-18T22:00:00-0700,2025-07-03T09:21:13-0700,2025-07-18T22:00:00-0700,2025-07-21T05:00:00-0700
14411,446035,ERS,W Beardsley Rd,West,Roadway closed on W Beardsley Rd Westbound fro...,1752901200,1751560525,1752901200,1.753099e+09,No Data,33.669810,-112.134810,33.668990,-112.169217,roadwork,Roadclosed,0,Major,icolEpilkTW|CSrCC^I`BGfBC|@Ab@?rA@x@@nCN`SVl^F...,None,None,None,None,None,None,None,"""<b>Mon:</b><br/>12:00 AM - 5:00 AM<br/><br/><...","[{""StartDate"": ""7/18/2025 10:00:00 PM-07:00:00...",Westbound Beardsley Road will be closed betwee...,3.0,2025-07-18T22:00:00-0700,2025-07-03T09:35:25-0700,2025-07-18T22:00:00-0700,2025-07-21T05:00:00-0700
14412,455326,ERS,I-10,East,Roadway reduced to two lanes on I-10 Eastbound...,1753066800,1752097175,1753066800,1.753099e+09,1 Right lane closed,32.341999,-111.071404,32.298251,-111.033833,roadwork,reduced2lanes,0,Major,oxkdEfs|dTtC{C~GyHdLmMb@c@hVuWbAaAzA_BhBuB|H{I...,25.0,None,None,None,45.0,None,None,"""<b>Mon:</b><br/>12:00 AM - 5:00 AM<br/><br/><...","[{""StartDate"": ""7/20/2025 8:00:00 PM-07:00:00""...",None,3.0,2025-07-20T20:00:00-0700,2025-07-09T14:39:35-0700,2025-07-20T20:00:00-0700,2025-07-21T05:00:00-0700
14413,455937,ERS,E Hunt Hwy,West,Roadway closed on E Hunt Hwy Westbound near ...,1752292800,1752180793,1752292800,1.753099e+09,All lanes closed,33.056480,-111.383710,NaN,NaN,closures,Roadclosed,1,Major,None,None,None,None,None,None,None,None,"""<b>Mon:</b><br/>12:00 AM - 5:00 AM<br/><br/><...","[{""StartDate"": ""7/11/2025 9:00:00 PM-07:00:00""...",Hunt Hwy closed at SR 79 for road construction...,1.0,2025-07-11T21:00:00-0700,2025-07-10T13:53:13-0700,2025-07-11T21:00:00-0700,2025-07-21T05:00:00-0700
14414,455942,ERS,SR-79,South,Turning lane closed on SR-79 Southbound from E...,1752292800,1752181217,1752292800,1.753099e+09,1 Right lane closed,33.060718,-111.379130,33.052121,-111.379133,roadwork,turninglaneclosed,0,Minor,odxhEpvxfTnl@?fG?,None,None,None,None,None,None,None,"""<b>Mon:</b><br/>12:00 AM - 5:00 AM<br/><br/><...","[{""StartDate"": ""7/11/2025 9:00:00 PM-07:00:00""...",Southbound right turn lane will be closed at H...,2.0,2025-07-11T21:00:00-0700,2025-07-10T14:00:17-0700,2025-07-11T21:00:00-0700,2025-07-21T05:00:00-0700


In [319]:
df['EncodedPolyline'].loc[[14410, 12585]]

14410    a~nlEtaskTbC?|A@B?
12585                  None
Name: EncodedPolyline, dtype: object

In [320]:

# df = df.loc[14410:14414]
for _, r in df.iterrows():
    ln = line_from_encoded(r.get('EncodedPolyline'))
    if ln is not None:
        geom.append(ln)
    else:
        if pd.notna(r['Longitude']) and pd.notna(r['Latitude']):
            geom.append(Point(r['Longitude'], r['Latitude']))
        else:
            geom.append(None)


In [321]:
gdf = gpd.GeoDataFrame(df, geometry=geom, crs='EPSG:4326')

light_props = [
    'ID','RoadwayName','DirectionOfTravel','EventType','EventSubType',
    'IsFullClosure','Severity', 'LocalReported', 'LocalStartDate','LocalPlannedEndDate','LanesAffected','LaneCount',
    'geometry'
]
gdf_map = gdf[light_props]

gdf_map


,ID,RoadwayName,DirectionOfTravel,EventType,EventSubType,IsFullClosure,Severity,LocalReported,LocalStartDate,LocalPlannedEndDate,LanesAffected,LaneCount,geometry
0,361395,US-60,West,roadwork,turninglaneclosed,0,Minor,2025-05-02T21:00:00-0700,2025-05-02T21:00:00-0700,2025-06-13T05:00:00-0700,No Data,3.0,"LINESTRING (-112.3571 33.64406, -112.35719 33...."
1,378561,4TH AVE,None,roadwork,road construction,0,None,2025-03-31T00:00:00-0700,2025-03-31T00:00:00-0700,2025-06-13T00:00:00-0700,No Data,NaN,POINT (-112.07893 33.4567)
2,378562,3RD AVE,None,roadwork,road construction,0,None,2025-03-31T00:00:00-0700,2025-03-31T00:00:00-0700,2025-06-13T00:00:00-0700,No Data,NaN,POINT (-112.07769 33.45669)
3,378620,24TH ST,None,roadwork,road construction,0,None,2025-04-28T21:00:00-0700,2025-04-28T21:00:00-0700,2025-06-13T05:00:00-0700,No Data,NaN,POINT (-112.03438 33.44106)
4,378642,INDIAN SCHOOL RD,None,roadwork,road construction,0,None,2025-05-06T00:00:00-0700,2025-05-06T00:00:00-0700,2025-06-13T00:00:00-0700,No Data,NaN,POINT (-112.05633 33.49472)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37215,506184,I-10,East,accidentsAndIncidents,C34exit,0,None,2025-09-10T13:15:00-0700,2025-09-10T13:15:00-0700,NaN,No Data,1.0,POINT (-110.88804 32.13474)
37216,506185,E TEXAS AVE / S INNOVATION WAY E MESA,None,accidentsAndIncidents,AccidentIncident,0,None,2025-09-10T13:17:49-0700,2025-09-10T13:17:49-0700,2149-05-03T06:39:59-0700,No Data,NaN,POINT (-111.67229 33.30758)
37217,506186,"N 59TH AV/W MCDOWELL RD ,PHX",None,accidentsAndIncidents,AccidentIncident,0,None,2025-09-10T13:15:18-0700,2025-09-10T13:15:18-0700,2149-05-03T06:39:59-0700,No Data,NaN,POINT (-112.18642 33.46597)
37218,506187,"PRIEST DR ,TMP",None,accidentsAndIncidents,AccidentIncident,0,None,2025-09-10T13:17:04-0700,2025-09-10T13:17:04-0700,2149-05-03T06:39:59-0700,No Data,NaN,POINT (-111.96312 33.34606)


In [323]:
gdf_preview = gdf_map.sample(500)

m = folium.Map(location=[33.4484, -112.0740], zoom_start=11)
folium.GeoJson(
    gdf_preview,
    name="WZDx (preview)",
    tooltip=folium.GeoJsonTooltip(fields=[f for f in ['RoadwayName','EventType', 'LocalReported', 'LocalStartDate', 'LocalPlannedEndDate']])
).add_to(m)
folium.LayerControl().add_to(m)
m.save("events.html")